# Prerequisites

In [1]:
%%capture

# Install dev version of the biomancy
!pip install -U git+https://github.com/hse-bioinflab/biomancy.git@dev

In [2]:
# Check that NVIDIA GPU is present
!nvidia-smi

Thu May 11 07:30:26 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.105.01   Driver Version: 515.105.01   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:4D:00.0 Off |                  Off |
|  0%   46C    P8    24W / 460W |     19MiB / 24564MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Configuration

## Paths

All paths used for training or inference are declared here. Some of them (like **FASTA** or **LABELS**) will need to be changed to point to the correct files.

In [3]:
from pathlib import Path

# Inputs
ROOT = Path.cwd()
DATA = ROOT / "data"

FASTA = DATA / "GRCh38.primary_assembly.genome.fa.gz"
BLACKLIST = DATA / "GRCh38-blacklist.bed" # Set to None if it's not available
LABELS = DATA / "Z-DNA.GRCh38.bed"
OMIC = DATA / "omic"

# Outputs
ARTIFACTS = ROOT / "artifacts"
ARTIFACTS.mkdir(exist_ok=True)

# Path for model whole-genome predictions
PREDICTION = ARTIFACTS / "whole-genome-predictions"
PREDICTION.mkdir(exist_ok=True)

In [4]:
# checking that everything is ok
for path in ROOT, DATA, FASTA, LABELS, OMIC, ARTIFACTS, PREDICTION:
    assert path.exists(), f"Path {path} doesn't exist!"

assert BLACKLIST is None or BLACKLIST.exists()

## Training / evaluation

In [5]:
import torch
import biomancy as bm

from multiprocessing import cpu_count


# See `Partition the genome` section for more details
WINDOW_SIZE = 1024
BIN_SIZE = WINDOW_SIZE * 100

# Batch size used to train/evaluate the model
BATCH_SIZE = {"train": 64, "val": 128, "test": 128}

# Data samplers used to train/evaluate the model
SAMPLERS = {
    "train": lambda dataset: bm.data.StratifiedGenomicSampler(dataset, BedTool(LABELS)),
    "val": lambda dataset: SequentialSampler(dataset),
    "test": lambda dataset: SequentialSampler(dataset),
}

# Number of threads used to prepare data batches
LOADER_THREADS = cpu_count()

# Device (GPU or CPU) for training & inference
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Data preprocessing

## Partition the genome

Divide the genome into large bins of **BIN_SIZE** base pairs, each of which is subsequently subdivided into windows of **WINDOW_SIZE** base pairs. All windows in one bin are used either for training, validation, or testing.

In [6]:
# Fetch chromosome lengths from the public UCSC database
# chromsizes = bm.data.derive.chromsizes(assembly="hg38")

# OR read chromosome lengths form the fasta index (use samtools faidx)
chromsizes = bm.data.derive.chromsizes(fasta=FASTA)

In [7]:
# Keep only primary chromosomes and collect their sizes
primary = set([f'chr{ind + 1}' for ind in range(22)] + ['chrX', 'chrY', 'chrM'])
chromsizes = {k: v for k, v in chromsizes.items() if k in primary}

In [7]:
from pybedtools import BedTool

# Exclude poorly assembled regions
EXCLUDE = bm.data.derive.ambiguous_sites(FASTA)

# Exclude blacklisted regions if available 
# Details: https://doi.org/10.1038/s41598-019-45839-z
if BLACKLIST and BLACKLIST.is_file():
    EXCLUDE = EXCLUDE.cat(
        BedTool(BLACKLIST).sort(), postmerge=True
    ).sort()

# Keep only 'selected' chromosomes
EXCLUDE = EXCLUDE.filter(lambda it: it.chrom in chromsizes)

In [8]:
from pybedtools import Interval

# Make bins & take care of the excluded regions
chromosomes = [Interval(chrom, 0, length, strand='+') for chrom, length in chromsizes.items()]
allgroups = bm.data.derive.bins(chromosomes, binsize=BIN_SIZE, exclude=EXCLUDE, dropshort=False)

# Partition into train-val-test bins
BINS = {}
trainval, BINS['test'] = bm.data.partition.randomly(allgroups, random_state=42)
BINS['train'], BINS['val'] = bm.data.partition.randomly(trainval, random_state=34)

# Split bins into final windows
WINDOWS = {}
for key, group in BINS.items():
    WINDOWS[key] = bm.data.derive.bins(group, binsize=WINDOW_SIZE, dropshort=True)

## Create datasets

Here we need to define all data sources used in the project (bed, fasta and bigwig files are supported). Each source defined here will be available in batches feed to the model with exactly the same keys as defined here. 

In this example we define the following sources:

- `targets`: target genome annotation in the BED format
- `ohe-sequence`: one-hot encoded genome sequence from the FASTA file
- `omics`: omics genome annotation in the BED format


Output shapes:
- `targets`: \[window size\]
- `ohe-sequence`: \[4, window size\]
- `omics`: \[files, window size\]

Note that omics genome annotations can be loaded from the [chip atlas](https://chip-atlas.org/) using the **data.load** module or manually from the [ENCODE](https://www.encodeproject.org/chip-seq-matrix).

In [9]:
SOURCES = {
    'targets': bm.data.sources.BED(LABELS, strand_specific=False, dtype='int64'),
    'ohe-sequence': bm.data.sources.fasta.OneHotEncoder(FASTA),
    'omics': bm.data.sources.concatenate(
        *[bm.data.sources.BED(path, strand_specific=False) for path in OMIC.glob('*.bed.gz')]
    )
}

Finally, combine the derived bins and data sources to get the final train/val/test datasets:

In [10]:
# Datasets
DATASETS = {key: bm.GenomicDataset(SOURCES, windows) for key, windows in WINDOWS.items()}

## Intervals augmentation

To artificially increase the size of the training datasets, we can apply the following transformations to our windows on the fly:

- `Shift`: move them a little (here by 50% of the window length)
- `RandomizeStrand` and randomize their strand

The probability of these transformations is controlled by the **pb** parameter. Can be "always", "never", or a number within the interval \[0, 1\].

We use `InjectLimits` to ensure that windows don't go beyond the parent bint.

In [11]:
DATASETS['train'].interval_transform = bm.T.Chain([
    bm.T.intervals.InjectLimits(rois=BINS['train']),
    bm.T.intervals.Shift(maxshift=0.5, pb='always'),
    bm.T.intervals.RandomizeStrand(pb='always')
])

## Create data loaders

[Data loaders](https://pytorch.org/docs/stable/data.html) are used to load data in parallel using multiple threads in the order specified by the [samplers](https://pytorch.org/docs/stable/data.html).

In [12]:
from torch.utils.data import DataLoader, SequentialSampler

LOADERS = {
    key: DataLoader(dataset, sampler=SAMPLERS[key](dataset), batch_size=BATCH_SIZE[key], num_workers=LOADER_THREADS)
    for key, dataset in DATASETS.items()
}

In [13]:
# Print how many batches we have for each partition
for key, loader in LOADERS.items():
    print(f'{key}\t->\t{len(loader)} batches')

test	->	133 batches
train	->	592 batches
val	->	99 batches


# Model

Here, we need to define the model whose parameters will be optimized during training. The model can include any differentiable functions, also known as layers, that are smooth and have a well-defined gradient. The model class must also inherit from the [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module) class to enable correct gradient calculations and tracing. You can find more information about available operations in the official [manual](https://pytorch.org/docs/stable/nn.html).

In [14]:
from torch import nn


class CNNModel(nn.Module):
    def __init__(self, in_features: int):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv1d(in_features, 12, 5, padding='same'),
            nn.ReLU(),
            nn.Conv1d(12, 12, 5, padding='same'),
            nn.ReLU(),
            nn.Conv1d(12, 2, 5, padding='same')
        )

    def forward(self, batch):
        inputs = torch.cat((batch['ohe-sequence'], batch['omics']), 1)
        logits = self.layers(inputs)
        proba = nn.functional.softmax(logits, dim=1)

        return {'proba': proba, 'logits': logits}

In [15]:
features = 4 + len(SOURCES['omics'].sources)
print('Total features:', features)

# Create the model and move to the target device (GPU)
MODEL = CNNModel(features).to(DEVICE)

Total features: 6


# Loss

Loss is a differentiable measure of how well the network is performing on a given task. Ideally, the loss is equal to the target metric, but that's not always possible because many metrics are not differentiable. The loss must produce a single value, which is then used to calculate gradients for all parameters (weights) of the model.

In [16]:
from torch.nn.functional import cross_entropy


class Loss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, logits, targets):
        loss = cross_entropy(logits, targets)
        return loss


LOSS = Loss()

# Training routines

## [Optimizer](https://pytorch.org/docs/stable/optim.html)
An optimizer is an algorithm that specifies how gradients will be used to adjust network parameters during training. It's normally called after each batch of data and applies gradients to all (or a subset) of network parameters, which theoretically should lead to minimizing the loss (the gradient's property).

Adam is a good default for many problems, you can read about it more [here](https://doi.org/10.48550/arXiv.1412.6980).

In [17]:
OPTIMIZER = torch.optim.Adam(MODEL.parameters(), lr=1e-3)

## [Engines](https://pytorch-ignite.ai/concepts/01-engine/)

Engine is an [ignite](https://pytorch-ignite.ai/) abstraction that loops a given number of times over provided data, executes a processing function and returns a result.

In [18]:
from ignite.engine import Engine


def train_step(engine: Engine, batch):
    OPTIMIZER.zero_grad()

    # Move all inputs to the target device
    batch = {k: v.to(DEVICE) for k, v in batch.items()}
    
    # Calculate the loss
    outputs = MODEL(batch)
    loss = LOSS(logits=outputs['logits'], targets=batch['targets'])
    
    # Calculate & apply gradients
    loss.backward()
    OPTIMIZER.step()

    return {"loss": loss, "targets": batch["targets"], "proba": outputs["proba"]}


# Train engine
TRAINER = Engine(train_step)

In [19]:
# Inference mode here is used to disable gradients calculation
@torch.inference_mode()
def eval_step(engine: Engine, batch):
    # Move all inputs to the target device
    batch = {k: v.to(DEVICE) for k, v in batch.items()}
    
    # Calculate outputs & the loss
    outputs = MODEL(batch)
    loss = LOSS(logits=outputs['logits'], targets=batch['targets'])

    return {"loss": loss, "targets": batch["targets"], "proba": outputs["proba"]}


# Validation engine
VALIDATOR = Engine(eval_step)

## Metrics

For the train, we will only report the smoothed loss value and nothing else. The reason is simple: train batches are drawn in a stratified, repetitive manner. So, each positive sample will be observed more than once per epoch. Moreover, each window is augmented before being drawn, so dataset changes a bit from epoch to epoch and results are not comparable.

We could run an additional engine to calculate metrics in deterministic non-extended train windows, but this is too computationally intensive and ultimately not very useful.

In [20]:
from ignite.metrics import RunningAverage

RunningAverage(alpha=0.98, output_transform=lambda r: r['loss']).attach(TRAINER, 'run_avg_train_loss')

For evaluation, we will focus on metrics that are calculated based on the confusion matrix. Namely - precision, recall, F1, and IOU.

Again, due to computational complexity, we will not calculate curve-based metrics such as ROC AUC or PR AUC.

In [21]:
from ignite.metrics import IoU, Loss
from ignite.metrics.confusion_matrix import ConfusionMatrix, cmPrecision, cmRecall, cmAccuracy

# Our target classes
CLASSES = ['bg', 'Z-DNA']

# Confusion matrix, it will be calculated only once
cm = ConfusionMatrix(
    num_classes=len(CLASSES),
    output_transform=lambda r: {'y_pred': r['proba'], 'y': r['targets']}
)

# Other metrics will be derived from the parent confusion matrix
cmbased = {
    "Precision": cmPrecision(cm, average=False),
    "Recall": cmRecall(cm, average=False),
    "IOU": IoU(cm),
}
cmbased["F1"] = 2 * cmbased["Precision"] * cmbased["Recall"] / \
                (cmbased["Precision"] + cmbased["Recall"] + 1e-15)

In [22]:
METRICS = {}

# Slice each metric to get per-cls values
for ind, cls in enumerate(CLASSES):
    for name, vec in cmbased.items():
        METRICS[f"{name}_{cls}"] = vec[ind]

# Calculate macro-averaged (mean) metric values (ignore 0 cls - bg)
for name, vec in cmbased.items():
    METRICS[f"m{name}"] = vec[1:].mean()

METRICS['loss'] = Loss(
    output_transform=lambda x: (x['loss'].unsqueeze(0), x['loss'].unsqueeze(0)),
    loss_fn=lambda *x: x[0][0]
)

Finally, we need to attach validation metrics to the validation engine:

In [23]:
for name, metric in METRICS.items():
    metric.attach(VALIDATOR, name)

## Logging
For logging metrics and loss values we will use [Tensorboard](https://www.tensorflow.org/tensorboard) (see below on how to view them after the training):

In [24]:
from ignite.contrib.handlers import TensorboardLogger
from ignite.handlers import global_step_from_engine
from ignite.engine import Events

logger = TensorboardLogger(log_dir=ARTIFACTS / "logs")

# Log validation metrics
logger.attach_output_handler(
    VALIDATOR,
    event_name=Events.EPOCH_COMPLETED,
    tag="val metrics",
    metric_names=list(METRICS.keys()),
    global_step_transform=global_step_from_engine(TRAINER)
)

# Log train loss
logger.attach_output_handler(
    TRAINER,
    event_name=Events.EPOCH_COMPLETED,
    tag="train loss",
    metric_names=['run_avg_train_loss']
)

## [Scheduler](https://pytorch.org/docs/stable/optim.html)

A scheduler is an algorithm that manipulates the learning rate (a parameter that scales gradient steps used by the optimizer) to improve the overall efficiency of the network training process. For example, if the learning rate is too high, the network may overshoot the optimal solution and fail to converge, while with a low learning rate, the network may take a long time to converge or get stuck in a suboptimal solution.

In this example, the scheduler will start with a high learning rate and then reduce it when no progress is observed for several batches in a row.

In [25]:
from ignite.handlers import LRScheduler, ReduceLROnPlateauScheduler

# Either torch scheduler
# scheduler = torch.optim.lr_scheduler.LRScheduler(OPTIMIZER, ...)
# scheduler = LRScheduler(scheduler)
# TRAINER.add_event_handler(Events.EPOCH_COMPLETED, scheduler)

# Or ignite-native scheduler
scheduler = ReduceLROnPlateauScheduler(
    OPTIMIZER, metric_name='run_avg_train_loss',
    mode="min", factor=0.35, patience=30, min_lr=1e-5,
    threshold_mode='rel', threshold=0.05,
)
TRAINER.add_event_handler(Events.EPOCH_COMPLETED, scheduler)

## Handlers

Handlers are called every epoch/iteration and are used to perform various tasks. Here we'll use them to display a progress bar, implement early stopping, and save model checkpoints:

In [26]:
from ignite.handlers import EarlyStopping, DiskSaver, Checkpoint
from ignite.contrib.handlers import ProgressBar

# Progress bar
ProgressBar(position=0, persist=True).attach(TRAINER, ['run_avg_train_loss'])
# ProgressBar(position=0, persist=True).attach(VALIDATOR, [])

# Early stopping
es = EarlyStopping(patience=100, score_function=lambda engine: engine.state.metrics['mF1'], trainer=TRAINER)
VALIDATOR.add_event_handler(Events.EPOCH_COMPLETED, es)

# Checkpointing
saver = DiskSaver(ARTIFACTS / "checkpoints", require_empty=False)

# - for train
to_save = {'model': MODEL, 'optimizer': OPTIMIZER, 'trainer': TRAINER}
ckpt = Checkpoint(
    to_save, saver, filename_prefix="train_", n_saved=2
)
TRAINER.add_event_handler(Events.EPOCH_COMPLETED(every=2), ckpt)

# - for validation
to_save = {'model': MODEL}
ckpt = Checkpoint(
    to_save,
    saver,
    filename_prefix="best",
    n_saved=1,
    score_function=Checkpoint.get_default_score_fn("mF1"),
    score_name="valid_mF1"
)
VALIDATOR.add_event_handler(Events.EPOCH_COMPLETED, ckpt)

/opt/conda/lib/python3.10/site-packages/ignite/contrib/handlers/tqdm_logger.py:127: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


# Train -> Val -> Test
Finally, we can use created engines to run the training and validation cycles:

In [27]:
# Validation will be evaluated every N train epochs
@TRAINER.on(Events.EPOCH_COMPLETED(every=4))
def _():
    VALIDATOR.run(LOADERS['val'])


TRAINER.run(LOADERS['train'], max_epochs=100)

Epoch [1/100]: [592/592] 100%|██████████████████████████████████████████████████████████████, run_avg_train_loss=0.0559 [00:03<00:00]
Epoch [2/100]: [592/592] 100%|██████████████████████████████████████████████████████████████, run_avg_train_loss=0.0403 [00:02<00:00]
Epoch [3/100]: [592/592] 100%|██████████████████████████████████████████████████████████████, run_avg_train_loss=0.0385 [00:02<00:00]
Epoch [4/100]: [592/592] 100%|██████████████████████████████████████████████████████████████, run_avg_train_loss=0.0372 [00:02<00:00]
Epoch [5/100]: [592/592] 100%|██████████████████████████████████████████████████████████████, run_avg_train_loss=0.0369 [00:02<00:00]
Epoch [6/100]: [592/592] 100%|██████████████████████████████████████████████████████████████, run_avg_train_loss=0.0378 [00:02<00:00]
Epoch [7/100]: [592/592] 100%|██████████████████████████████████████████████████████████████, run_avg_train_loss=0.0375 [00:02<00:00]
Epoch [8/100]: [592/592] 100%|████████████████████████████████

State:
	iteration: 59200
	epoch: 100
	epoch_length: 592
	max_epochs: 100
	output: <class 'dict'>
	batch: <class 'dict'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

## Tensorboard

You can check metrics logged during training/validation with [tensorboard](https://pypi.org/project/tensorboard/). 

To do this you will need a separate SSH connection with port forwarding:
```shell
ssh -L localhost:6006:localhost:6006 username@server
cd DL-template
tensorboard --port 6006 --logdir artifacts/logs
```
The session will be available [here](http://127.0.0.1:6006).

## Test

We will test the model that showed the best performance on the validation. It should be saved in the **checkpoints** folder, paste its name instead of the ``"best_model_valid_mF1=XXXXX.pt"``

In [29]:
# Load the best model
Checkpoint.load_objects(
    to_load={"model": MODEL},
    checkpoint=ARTIFACTS / "checkpoints" / "best_model_valid_mF1=0.2012.pt"
)

In [30]:
# Create test-engine
TESTER = Engine(eval_step)

# Attach required metrics
for name, metric in METRICS.items():
    metric.attach(TESTER, name)

In [31]:
# Evaluate the model
state = TESTER.run(LOADERS['test'])

In [32]:
# Print test metrics
for metric, value in state.metrics.items():
    print(f"{metric}\t->\t{value:.3f}")

Precision_bg	->	1.000
Recall_bg	->	0.999
IOU_bg	->	0.999
F1_bg	->	0.999
Precision_Z-DNA	->	0.140
Recall_Z-DNA	->	0.298
IOU_Z-DNA	->	0.106
F1_Z-DNA	->	0.191
mPrecision	->	0.140
mRecall	->	0.298
mIOU	->	0.106
mF1	->	0.191
loss	->	0.006


# Whole-genome predictions

To create whole genome predictions, we will use the `serve` module. As with engines, you need to declare a function that receives a batch from the data sources and outputs the probabilities/arrays that you want to store.

In [33]:
from biomancy import serve


# A function to generate model predictions
def predictor(batch):
    # Move all features to the target device
    batch = {k: v.to(DEVICE) for k, v in batch.items()}
    # Calculate probabilities and transform them to a numpy ndarray
    proba = MODEL(batch)['proba'].cpu().numpy()
    # You can return any values here
    return {"proba": proba}


## Strategy
The strategy outlines how to divide input intervals into windows and handle boundary cases.

In this case, we merge input intervals and divide them into overlapping windows. We use only the central 90% of predictions for each bin because predictions near the edges are less accurate. Example:

In [34]:
strategy = serve.strategy.MergeAndBin(WINDOW_SIZE, roisize=0.9)

## Output hooks
Output hooks are used to save model predictions in different formats. They are called for each predicted window with the following parameters:

* `it`(Interval) - the bin for which predictions were generated  
* `roi` (int, int) - part of the input bin (start/end coordinates) where the predictions are reliable  
* `inps` (dict) - input features  
* `prep` (dict) - values returned by the predictor function  

We should use adapters to transform this data into writable records (see below).

### BigWig
* `quantlvl`: quantize output predictions to these levels to save space (i.e., all predictions will be rounded up to these values)
* `skip_below`: skip predictions where probability < this threshold.

Here, the adapter passes the output of the predictor to the `BigWig.Record` constructor to create write-ready bigwig records. No complicated logic, the adapter is only needed to select the output array we want to store. 

In [35]:
import numpy as np

quantlvl = np.arange(0.1, 1.1, step=0.1)
skip_below = 0.1

bw = serve.io.BigWig(
    chromsizes, PREDICTION / "probability.bw",
    adapter=lambda it, roi, _, pred: serve.io.BigWig.Record.from_array(
        it, pred['proba'][1, :], roi=roi, quantlvl=quantlvl, skip_below=skip_below
    )
)

### BED
* `thr`: the threshold at which the result is considered positive and will be written to the bed file 

As above, the adapter is only needed to extract the array of target probabilities from the `predictor` function output.

In [36]:
thr = 0.75

bed = serve.io.BED(
    PREDICTION / "peaks.bed",
    adapter=lambda it, roi, _, pred: serve.io.BED.Record.from_array(it, pred['proba'][1,:], roi=roi, thr=0.75)
)

## Inference

Finally, here's how we can run the prediction:

In [37]:
serve.run(
    predictor, SOURCES, chromosomes, hooks=(bw, bed), strategy=strategy,
    # data loader parameters
    num_workers=LOADER_THREADS, batch_size=BATCH_SIZE['test']
)

When completed, the predictions will be saved in the **PREDICTION** folder. No additional steps are required.


Optionally, we can change the read/write permissions of the artifacts folder to allow their full use outside of the container:

In [38]:
!chmod -R a+rw {ARTIFACTS}